In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
train = pd.read_csv("../data/raw_data/train.csv")
train.head(2)

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree


In [64]:
def add_special_tokens(marker_type, tokenizer):
    """토크나이저에 special tokens를 추가

    Args:
        marker_type (type): 적용하고자 하는 marker_type
        tokenizer (Tokenizer): 사용하는 토크나이저

    Returns:
        추가한 토큰 갯수, 토크나이저
    """
    if marker_type == "entity_marker":
        markers = ["[E1]", "[/E1]", "[E2]", "[/E2]"]
        added_token_num = tokenizer.add_special_tokens({"additional_special_tokens": markers})

    elif marker_type == "entity_marker_punc":
        added_token_num = 0

    elif marker_type == "typed_entity_marker":
        entity_types = ["PER", "ORG", "POH", "DAT", "LOC", "NOH"]
        markers = []

        for type in entity_types:
            markers.extend((f"<S:{type}>", f"</S:{type}>", f"<O:{type}>", f"</O:{type}>"))
        added_token_num = tokenizer.add_special_tokens({"additional_special_tokens": markers})
    
    elif marker_type == "typed_entity_marker_punc_1":
        added_token_num = 0
    
    elif marker_type == "typed_entity_marker_punc_2":
        added_token_num = 0
    
    elif marker_type == "typed_entity_marker_punc_3":
        markers = ['PER', 'ORG', 'POH', 'DAT', 'LOC', 'NOH']
        added_token_num = tokenizer.add_special_tokens({"additional_special_tokens": markers})

    return added_token_num, tokenizer


In [65]:
def mark_entity(marker_type, word, word_type, is_subj):  
    """입력한 word에 marker_type에 따른 special token 추가

    Args:
        marker_type (str): 적용하고자 하는 marker_type
        word (str): entity 단어
        word_type (str): entity 단어의 유형
        is_subj (bool): subj entity이면 True, obj entity이면 False

    Returns:
        entity 단어에 marker type에 따른 special token로 감싼 문자열
    """
    if marker_type == "entity_marker":
        marked_word = f" [E1] {word} [/E1] " if is_subj else f" [E2] {word} [/E2] "

    elif marker_type == "entity_marker_punc":
        marked_word = f" @ {word} @ " if is_subj else f" # {word} # "

    elif marker_type == "typed_entity_marker":
        marked_word = f" <S:{word_type}> {word} </S:{word_type}> " if is_subj else f" <O:{word_type}> {word} </O:{word_type}> "

    elif marker_type == "typed_entity_marker_punc_1":
        entity_types = {'ORG':'기관명', 'PER':'인명', 'POH':'기타', 'DAT':'날짜', 'LOC':'지명', 'NOH':'수량'}
        marked_word = f" @ * {entity_types[word_type]} * {word} @ " if is_subj else f" # ^ {entity_types[word_type]} ^ {word} # "
    
    elif marker_type == "typed_entity_marker_punc_2": # PER special token으로 추가하지 않고 사용
        marked_word = f" @ * {word_type} * {word} @ " if is_subj else f" # ^ {word_type} ^ {word} # "
    
    elif marker_type == "typed_entity_marker_punc_3": # ORG special token으로 추가하고 사용
        marked_word = f" @ * {word_type} * {word} @ " if is_subj else f" # ^ {word_type} ^ {word} # "

    return marked_word

In [66]:
def get_entity_marked_dataframe(marker_type, df):

    """maker_type을 적용한 데이터프레임을 반환

    Args:
        marker_type (str): 적용하고자 하는 marker_type
        df (DataFrame): 원본 데이터프레임

    Returns:
        marker_type이 적용된 데이터프레임
    """
    assert marker_type in ["entity_marker", "entity_marker_punc","typed_entity_marker", "typed_entity_marker_punc_1", "typed_entity_marker_punc_2", "typed_entity_marker_punc_3"], "marker type은 [entity_marker, entity_marker_punc, typed_entity_marker, typed_entity_makrer_punc_1~3] 중에서 사용이 가능합니다."

    df_entity_marked = pd.DataFrame(columns = ["id" , "sentence", "subject_entity", "object_entity", "label", "source"])

    for i in tqdm(range(len(df)), total=(len(df))):
        id, sentence, subj, obj, label, source = df.iloc[i]["id"], df.iloc[i]["sentence"], df.iloc[i]["subject_entity"], df.iloc[i]["object_entity"], df.iloc[i]["label"], df.iloc[i]["source"]

        subj_word, subj_start, subj_end, subj_type = eval(subj).values()
        obj_word, obj_start, obj_end, obj_type = eval(obj).values()

        try:
            # subject
            subj_start_idx = sentence.index(subj_word)
            subj_end_idx = subj_start_idx + len(subj_word) - 1 
            
            # object
            obj_start_idx = sentence.index(obj_word)
            obj_end_idx = obj_start_idx + len(obj_word) - 1 
        except Exception:
            print("Not Found")

        if subj_end_idx < obj_start_idx:
            sentence = sentence[:subj_start_idx] + mark_entity(marker_type, subj_word, subj_type, is_subj=True) + sentence[subj_end_idx+1:obj_start_idx] + mark_entity(marker_type, obj_word, obj_type, is_subj=False) + sentence[obj_end_idx+1:]
        elif obj_end_idx < subj_start_idx:
            sentence = sentence[:obj_start_idx] + mark_entity(marker_type, obj_word, obj_type, is_subj=False)  + sentence[obj_end_idx+1:subj_start_idx] + mark_entity(marker_type, subj_word, subj_type, is_subj=True) + sentence[subj_end_idx+1:]


        df_entity_marked.loc[i] = [id, sentence, subj, obj, label, source]

    return df_entity_marked

In [67]:
test_df = train[:4]

In [68]:
test_df

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree


In [73]:
temp = get_entity_marked_dataframe("typed_entity_marker_punc_3", test_df)

100%|██████████| 4/4 [00:00<00:00, 453.03it/s]


In [74]:
temp

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 # ^ PER ^ 조지 해리슨 # 이 쓰고 @ * ORG...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당· # ^ ORG ^ 대안신당 # · @ * ORG * 민주...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 @ * ORG * 광주FC @ 는 지난 26...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주) @ * ORG * 아성다이소 @ (대표 # ^ PER ^...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree


['〈', 'So', '##me', '##th', '##ing', '〉', '는', '[', 'E', '##2', ']', '조지', '해리', '##슨', '[', '/', 'E', '##2', ']', '이', '쓰', '##고', '[', 'E', '##1', ']', '비틀즈', '[', '/', 'E', '##1', ']', '가', '1969', '##년', '앨범', '《', 'Ab', '##be', '##y', 'Ro', '##ad', '》', '에', '담', '##은', '노래', '##다', '.']

['〈', 'So', '##me', '##th', '##ing', '〉', '는', '[E2]', '조지', '해리', '##슨', '[/E2]', '이', '쓰', '##고', '[E1]', '비틀즈', '[/E1]', '가', '1969', '##년', '앨범', '《', 'Ab', '##be', '##y', 'Ro', '##ad', '》', '에', '담', '##은', '노래', '##다', '.']

['〈', 'So', '##me', '##th', '##ing', '〉', '는', '#', '조지', '해리', '##슨', '#', '이', '쓰', '##고', '@', '비틀즈', '@', '가', '1969', '##년', '앨범', '《', 'Ab', '##be', '##y', 'Ro', '##ad', '》', '에', '담', '##은', '노래', '##다', '.']

<br/>

['〈', 'So', '##me', '##th', '##ing', '〉', '는', '#', '조지', '해리', '##슨', '#', '이', '쓰', '##고', '@', '비틀즈', '@', '가', '1969', '##년', '앨범', '《', 'Ab', '##be', '##y', 'Ro', '##ad', '》', '에', '담', '##은', '노래', '##다', '.']

['〈', 'So', '##me', '##th', '##ing', '〉', '는', '<', 'O', ':', 'PER', '>', '조지', '해리', '##슨', '<', '/', 'O', ':', 'PER', '>', '이', '쓰', '##고', '<', 'S', ':', 'O', '##R', '##G', '>', '비틀즈', '<', '/', 'S', ':', 'O', '##R', '##G', '>', '가', '1969', '##년', '앨범', '《', 'Ab', '##be', '##y', 'Ro', '##ad', '》', '에', '담', '##은', '노래', '##다', '.']


['〈', 'So', '##me', '##th', '##ing', '〉', '는', '<O:PER>', '조지', '해리', '##슨', '</O:PER>', '이', '쓰', '##고', '<S:ORG>', '비틀즈', '</S:ORG>', '가', '1969', '##년', '앨범', '《', 'Ab', '##be', '##y', 'Ro', '##ad', '》', '에', '담', '##은', '노래', '##다', '.']